In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import dirichlet, beta, norm
from numpy.random import choice
from pandas import Series
np.set_printoptions(precision=2)

A large portion of the materials/code in this notebook are adapted or taken from: [TIM HOPPER](https://dp.tdhopper.com/)

# Dirichlet Distribution

The Dirichlet distribution can be considered as a distribution of distributions. Each sample from the Dirichlet distribution is a categorical distribution over $K$ categories. The distribution is parameterized using a $K$ dimensional vector $\alpha$. This parameter is also known as the concentration parameter. We can express the probability density function of the Dirichlet distribution as follows:

$$f(x_1, \dots, x_K; \alpha_1, \dots, \alpha_K) = \frac{1}{B(\alpha)} \prod_{i=1}^{K}x_{i}^{\alpha_{i} - 1}$$

where $\{x_{i}\}_{i=1}^{i=K}$ belong to the $K-1$ [simplex](https://en.wikipedia.org/wiki/Simplex). So $\sum_{i=1}^{K}x_{i} = 1$ and $x_{i} \geq 0 \ \forall i \in [1,K]$.

Here $B(\alpha)$ refers to the multivariate [Beta function](https://en.wikipedia.org/wiki/Beta_function) which we can express in terms of the [Gamma function](https://en.wikipedia.org/wiki/Gamma_function) as:
$$B(\alpha) = \frac{\prod_{i=1}^{K}\Gamma(\alpha_i)}{\Gamma(\sum_{i=1}^{K}a_i)}$$

The expected value of the Dirichlet distribution depends on the $\alpha$ vector and is given by:

$$\mathbb{E}[(x_{i}, \dots, x_{K})] = \frac{(a_{1}, \dots, a_{K})}{\sum_{i=1}^{K}a_{i}}$$

As the value (i.e. magnitude) of the concentration parameter $\alpha$ increases the standard deviation of the distribution decreases.

In [ ]:
def stats(scale_factor, G0=[1, 1, 1], N=10000):
    samples = dirichlet(alpha = scale_factor * np.array(G0)).rvs(N)
    print("                          alpha:", scale_factor)
    print("              element-wise mean:", samples.mean(axis=0))
    print("element-wise standard deviation:", samples.std(axis=0))
    print()

In the example below we draw 10000 samples from a symmetric Dirichlet distribution (i.e. one where each component of the $\alpha$ vector is equal).

Notice that as we increase the concentration parameter $\alpha$ the standard deviation of our distribution decreases. The mean value of the distribution remains thesame however.

In [ ]:
for alpha_val in [0.1, 1, 10, 100, 1000]:
    stats(alpha_val)

# Dirichlet Process

The Dirichlet Process can be seen as an infinite-dimensional generalization of the Dirichlet distribution. More concretely, in the same way as the Dirichlet distribution is the conjugate prior for the categorical distribution, the Dirichlet process is the conjugate prior for the infinite, nonparametric discrete distributions. For example, Dirichlet processes is a prior probability distribution for infinite mixture models.

A Dirichlet process is parameterized by a base probability distribution $H$ and a positive real number $\alpha$ and in general generates samples which are distributions similar to $H$. $H$ is also referred to as a base measure.

A sample $G$ (recall that $G$ is also a probability distribution) from a Dirichlet process, $\textrm{DP}(\alpha, H)$, can be constructed by drawing a countable infinity of samples $\theta_{k}$ from $H$. We define $G \sim \textrm{DP}(\alpha, H)$ as:

$$G = \sum_{k=1}^{\infty} \pi_{k} \delta_{\theta_k}$$

$\pi_k$ are weights that sum to 1 (those weights as we shall later see are also derived from a Dirichlet distribution) and $\delta$ is the Dirac delta function.

Notice that $G$ is a *discrete* distribution that takes the value $\theta_k$ with probability $\pi_k$. This is true regardless of if $H$ has a continuous support (a support of a distribution is defined as the set of possible values of a random variable in a distribution). The support of $G$ is a countable infinite subset of the support of $H$.

## Defining Property of the Dirichlet Process
[Gregor Heinrich](http://www.arbylon.net/publications/ilda.pdf) writes:

<font size="5">
The defining property of the DP is that its samples have weights $\pi_k$ and locations $\theta_k$ distributed in such a way that when partitioning $S(H)$ into finitely many arbitrary disjoint subsets $S_1, \dots, S_j$ for $J<\infty$, the sums of the weights $\pi_k$ in each of these $J$ subsets are distributed according to a Dirichlet distribution described by $\mathrm{Dir}(\alpha G_0)$ where $G_0$ is a discrete base distribution whose weights are equal to the integrals of the base distribution $H$ over the subsets $S_n$.
</font>

<br />
<br />

For example consider a Dirichlet process with a standard normal base measure $H \sim \mathcal{N}(0, 1)$. Let $H$ be a sample from $\mathrm{DP}(\alpha, H)$ and partition the real line (the support of a normal distribution) as $S_1=(-\infty,-1], S_2=(-1,1], and S_3=(1,\infty]$ then

$$H(S_1),H(S_2),H(S_3)\sim Dir(\alpha \ \mathrm{erf}(-1), \alpha (\ \mathrm{erf}(1) - \mathrm{erf}(-1)),\alpha(1 - \mathrm{erf}(1)))$$

Note that $H(S_n)$ denotes the sum of the $\pi_k$ values whose $\theta_k$ lie in the partition $S_n$. $\mathrm{erf}$ refers to the error function which can define integral segments over a Gaussian distribution. This partitioning property holds for any possible partitions over the support of the base measure.

**This means that for any sample from a Dirichlet process we can construct a sample from a Dirichlet distribution by partitioning the support of the sample into finite number of bins.**

## Choosing the weights $\pi$ (Stick Breaking Construction)

To satisfy the above property there are various equivalent ways to generate the weights $\pi_k$ values. Some notable procedures are the Chinese restaurant process, the Polya urn scheme and the Stick breaking construction which we discuss here.

Given a stick of length 1 at the beginning of each iteration $k$, we repeatedly break off a fraction $\beta_{k}$ of the remaining stick. $\beta_{k} \in [0,1]$ is sampled from a $Beta(1, \alpha)$ distribution. The weights $\pi_{k}$ are defined as the actual length of the stick removed at iteration $k$. So initially $\beta_1 = \pi_1$. More generally we have:

$$\pi_k = \beta_k \prod_{j=1}^{k-1}(1-\beta_j)$$

So one naive way to draw a sample for a Dirichlet process is to sample an infinite number of $\theta_k$ values from the base distribution $H$ and infinite number of $\beta_k$ values from the $Beta(1,\alpha)$ in order to construct our $\pi_k$. 

Notice that the weights $\pi_k$ are always positive and get progressively smaller as $k\rightarrow \infty$. Also notice that $\sum_{k=1}^{\infty} \pi_{k} = 1$. So this means we can approximate $G \sim \mathrm{DP}(\alpha, H)$ with a sufficient amount of samples from $H$ quite accurately. 

## Visualizing the Dirichlet Process

Below we use this stick breaking construction to draw approximate samples from several Dirichlet processes with a standard normal, $\mathcal{N}(0,1)$, base distribution, $H$, but varying $\alpha$ parameter.

Recall that a single sample from a Dirichlet process is a probability distribution over a countably infinite subset of the support of the base measure.

The blue line is the PDF(Probability Density Function) for a standard normal. The black lines represent the $\theta_k$ and $\pi_k$ values; $\theta_k$ is indicated by the position of the black line on the x-axis; $\pi_k$ is proportional to the height of each line.

We generate enough $pi_k$ values are generated so their sum is greater than 0.99. When $\alpha$ is small, very few $\theta_k$'s will have corresponding $\pi_k$ values larger than 0.01. However, as $\alpha$ grows large, the sample becomes a more accurate (though still discrete) approximation of $\mathcal{N}(0,1)$.

In [ ]:
def dirichlet_sample_approximation(base_measure, alpha, tol=0.01):
    '''
    Extract respective pi and theta values for a dirichlet process using the stick breaking construction
    This is an approximation such that the pi values sum to more than 1-tol. 
    '''
    betas = []
    pis = []
    betas.append(beta(1, alpha).rvs())
    pis.append(betas[0])
    
    while sum(pis) < (1.-tol):
        s = np.sum([np.log(1 - b) for b in betas])
        new_beta = beta(1, alpha).rvs() 
        betas.append(new_beta)
        pis.append(new_beta * np.exp(s))
    pis = np.array(pis)
    thetas = np.array([base_measure() for _ in pis])
    return pis, thetas

def plot_normal_dp_approximation(alpha):
    fig, ax1 = plt.subplots(figsize=(12,8))
    fontsize = 20
    
    ax2 = ax1.twinx()
    ax1.set_ylabel("$\pi$ value", fontsize=fontsize)
    ax2.set_ylabel("Base Measure probability density", fontsize=fontsize)
    fig.suptitle("Dirichlet Process Sample with N(0,1) Base Measure \n $alpha$="+str(alpha), fontsize=1.2*fontsize)
    
    # Sample from base measure and calculate pi values
    pis, thetas = dirichlet_sample_approximation(lambda: norm().rvs(), alpha)
    ax1.vlines(thetas, 0, pis, )

    # Plot the Base measure
    X = np.linspace(-4,4,100)
    ax2.plot(X, norm.pdf(X))
    
    plt.show()

In [ ]:
plot_normal_dp_approximation(.1)
plot_normal_dp_approximation(1)
plot_normal_dp_approximation(10)
plot_normal_dp_approximation(1000)

Often we want to draw samples from a distribution sampled from a *Dirichlet process* instead of from the Dirichlet process itself. Much of the literature on the topic refers to this as sampling from a Dirichlet process.

Fortunately, we don't have to draw an infinite number of samples from the base distribution and stick breaking process to do this. Instead, we can draw these samples as they are needed.

Suppose, for example, we know a finite number of the $\theta_k$ and $\pi_k$ values for a sample $G \sim \textrm{Dir}(\alpha, H)$. For example, we know

$$\pi_1=0.5,\pi_2=0.3,\theta_1=0.1,\theta_2=-0.5.$$

To sample from $G$, we can generate a uniform random u number between 0 and 1. If $u$ is less than 0.5, our sample is 0.1. If $0.5\leq u<0.8$, our sample is $-0.5$. If $u\geq 0.8$, our sample (from $G$ will be a new sample $\theta_3$ from $H$. At the same time, we should also sample and store $\pi_3$. When we draw our next sample, we will again draw $u \sim \textrm{Uniform}(0,1)$ but will compare against $\pi_1,\pi_2$ AND $pi_3$.

The class below will take a base distribution $H$ and $\alpha$ as arguments to its constructor. The class instance can then be called to generate samples from $G \sim \textrm{DP}(\alpha,  H)$.

In [ ]:
class DirichletProcessSample():
    '''
    Given a base measure and an alpha value return a sample from a distribution
    sampled from a Dirichlet process.
    '''
    def __init__(self, base_measure, alpha):
        self.base_measure = base_measure
        self.alpha = alpha
        
        self.cache = []
        self.weights = []
        self.total_stick_used = 0.

    def __call__(self):
        remaining = 1.0 - self.total_stick_used
        i = DirichletProcessSample.roll_die(self.weights + [remaining])
        if i is not None and i < len(self.weights):
            return self.cache[i]
        else:
            stick_piece = beta(1, self.alpha).rvs() * remaining
            self.total_stick_used += stick_piece
            self.weights.append(stick_piece)
            new_value = self.base_measure()
            self.cache.append(new_value)
            return new_value
        
    @staticmethod 
    def roll_die(weights):
        if weights:
            return choice(range(len(weights)), p=weights)
        else:
            return None

In [ ]:
base_measure = lambda: norm().rvs()
n_samples = 20000
samples = {}
for alpha in [1, 10, 100, 1000]:
    dirichlet_norm = DirichletProcessSample(base_measure=base_measure, alpha=alpha)
    samples["Alpha: %s" % alpha] = [dirichlet_norm() for _ in range(n_samples)]

_ = pd.DataFrame(samples).hist(figsize=(15,15), bins=40)

Note that these histograms look very similar to the corresponding plots of sampled distributions above. However, these histograms are plotting points sampled from a distribution sampled from a Dirichlet process while the plots above were showing approximate distributions samples from the Dirichlet process. Of course, as the number of samples from each $G$ grows large, we would expect the histogram to be a very good empirical approximation of $G$.

# Hierarchical Dirichlet Process (HDP)

HDP is a nonparametric Bayesian approach to cluster grouped data. In HDP a Dirichlet process is used for each group of data with the Dirichlet processes for all groups sharing the same base distribution which is itself drawn from a Dirichlet process.

---

To illustrate this definition let's consider a topic model for words in a set of documents (i.e. corpus). The groups (i..e the bag of words in a document) in the corpus are indexed by $j = 1, \dots , J$ and each group is consisted of data items (i.e. words) as $x_{j1}, \dots, x_{jn}$.

The HDP is parameterized by a base distribution $H$ that governs the a priori distribution over data items, and a number of concentration parameters that govern the a priori number of clusters and amount of sharing across groups. The $j$-th group is associated with a random probability measure $G_{j}$ which has distribution given by a Dirichlet process:

$$G_j | G_0 \sim \textrm{DP}(a_j, G_0)$$

$a_j$ denotes the concentration parameter associated with the group $j$ and $G_0$ is the base distribution that is shared across all groups. The common distribution is itself distribution drawn from a Dirichlet process as:

$$G_0 \sim \textrm{DP}(a_0, H)$$

Where $a_0$ is the concentration parameter and $H$ is is the base distribution.

Each data item $x_{ji}$ is associated with a latent parameter $\theta_{ji}$ as:

$$\theta_{ji} | G_j \sim G_j$$
$$x_{ji} | \theta_{ji} \sim F(\theta_{ji})$$

This means that each latent parameter $\theta_{ji}$ has a prior distribution given by $G_j$ and each data item $x_{ji}$ has a distribution $F(\theta_{ji})$ parameterized by its respective latent parameter.

## Sampling from a Hierarchical Dirichlet Process

Let's assume we have as our base measure $H$ the standard normal distribution.

If we sample from the normal distribution we would expect that every time we sample a unique value since the normal distribution has a continuous support. Below we sample 10 thousand samples from it and note the number of unique values samples.

In [ ]:
base_measure = lambda: norm().rvs() 

ndraws = 10000
print("Number of unique samples after", ndraws, "draws:")
draws = Series([base_measure() for _ in range(ndraws)])
print(draws.unique().size)

However, if we sample 10 thousand samples from a Dirichlet Process that has as base measure $H$ then we start seeing duplicate samples since the Dirichlet Process is descrete. Moreover, the number of unique samples goes up as $\alpha$ increases.

In [ ]:
norm_dp = DirichletProcessSample(base_measure, alpha=100)

print("Number of unique samples after", ndraws, "draws:")
dp_draws = Series([norm_dp() for _ in range(ndraws)])
print(dp_draws.unique().size)

Notice that `dp_draws` returns samples from a probability distribution (specifically, a probability distribution sampled from $\textrm{DP}(\alpha, H_0)$). So we can use `dp_draws` as a base distribution for another Dirichlet process!

In [ ]:
norm_hdp = DirichletProcessSample(norm_dp, alpha=10)

What does this mean?

We know that `norm_dp` is a sampler from a probability distribution that looks like the standard normal distribution which we denoted as $H$. `norm_hdp` is a sampler from a probability distribution that "looks like" the distribution `norm_dp` samples from.

Here is a histogram of 10k samples drawn from `norm_dp`, our first sampler.

In [ ]:
Series(norm_dp() for _ in range(10000)).hist(figsize=(10,10), bins=40)
_=plt.title("Histogram of Samples from norm_dp", fontsize=20)

And here is a histogram for samples drawn from `norm_hdp`, our second sampler.

In [ ]:
Series(norm_hdp() for _ in range(10000)).hist(figsize=(10,10), bins=40)
_=plt.title("Histogram of Samples from norm_hdp", fontsize=20)

Notice how the second plot doesn't look very much like the first! The level to which a sample from a Dirichlet process approximates the base distribution is a function of the dispersion parameter $\alpha$. Because we set $\alpha=10$ (which is relatively small) for the HDP, the approximation is fairly course. A small $\alpha$ value means that we will more frequently reuse values already seen instead of drawing new ones.

Now we can define a Python class to encapsulate a sample from the Hierarchical Dirichlet Process as a base class of the Dirichlet process.

In [ ]:
class HierarchicalDirichletProcessSample(DirichletProcessSample):
    def __init__(self, base_measure, alpha1, alpha2):
        first_level_dp = DirichletProcessSample(base_measure, alpha1)
        self.second_level_dp = DirichletProcessSample(first_level_dp, alpha2)

    def __call__(self):
        return self.second_level_dp()

Since the Hierarchical DP is a Dirichlet Process inside of Dirichlet process, we must provide it with both a first and second level 

In [ ]:
norm_hdp = HierarchicalDirichletProcessSample(base_measure, alpha1=10, alpha2=20)

We can sample directly from the probability distribution drawn from the Hierarchical Dirichlet Process.

In [ ]:
Series(norm_hdp() for _ in range(10000)).hist(figsize=(10,10), bins=40)
_=plt.title("Histogram of samples from distribution drawn from Hierarchical DP", fontsize=20)

Notice that `norm_hdp` is not equivalent to the Hierarchical Dirichlet Process; it samples from a single distribution sampled from this HDP. Each time we instantiate the `norm_hdp` variable, we are getting a sampler for a unique distribution. Below we sample five times and get five different distributions.

In [ ]:
for i in range(5):
    norm_hdp = HierarchicalDirichletProcessSample(base_measure, alpha1=10, alpha2=10)
    _=Series(norm_hdp() for _ in range(100)).hist(figsize=(6,6), bins=20)
    _=plt.title("Histogram of samples from distribution drawn from Hierarchical DP", fontsize=15)
    _=plt.figure()